# Explorando as estatísticas brasileiras de comércio exterior

Vamos começar carregando o pacote Pandas e algumas tabelas auxiliares.

In [ ]:
import pandas as pd

### Tabelas auxiliares
nome_sh = pd.read_csv('data/nome_sh.csv', 
                      sep = ';',
                      dtype = {'co_sh2': 'str'})


nome_pais = pd.read_csv('data/nome_pais.csv', 
                        sep = ';',
                        dtype = {'co_pais': 'str'})

A tabela **nome_sh** apresenta os nomes dos Capítulos (dois dígitos) do Sistema Harmonizado, classificação de mercadorias muito usada no comércio internacional. 

A tabela **nome_pais** traz os códigos de três dígitos de país usados nas bases de dados da Secretaria de Comércio Exterior. 

Dessa tabela, por hoje nos interessamos somente pelos códigos e nomes dos países em português. Exclua as demais colunas do DataFrame **nome_pais**.

In [ ]:
nome_pais = nome_pais.loc[:, ['co_pais', 'no_pais']]

Carregamos a seguir os dados de exportação de 2022, com valores em dólares americanos (**vl_fob**) agregados por país de destino e produto no nível do subitem (8 dígitos) da Nomenclatura Comum do Mercosul (NCM).

In [ ]:
exp_2022 = pd.read_csv("data/exp_2022.csv", 
                       sep = ";",
                       dtype = {'co_ncm': 'str', 'co_pais': 'str'})

In [ ]:
exp_2022

Vamos transportar os códigos do nível mais desagregado (subitem NCM, de oito dígitos) para o mais agregado (Capítulo SH, de dois dígitos). 

O Capítulo SH corresponde aos dois primeiros dígitos do código NCM.

Crie a coluna **co_sh2** no DataFrame **exp_2022**.

In [ ]:
exp_2022 = exp_2022.assign(co_sh2 = lambda df: df['co_ncm'].str.slice(0, 2))

In [ ]:
exp_2022

Identifique os produtos mais exportados no ano passado, agregando os valores (**vl_fob**) por Capítulo SH. 

Inclua a correspondência dos nomes com os códigos de produto na mesma tabela, e ordene-a de forma decrescente pelo valor exportado.

In [ ]:
principais_produtos = (exp_2022
                      .groupby(['co_sh2'], as_index = False)
                      .agg({'vl_fob': 'sum'})
                      .merge(nome_sh, on = 'co_sh2')
                      .sort_values('vl_fob', ascending = False)
                      .loc[:, ['co_sh2', 'no_sh2_por', 'vl_fob']]
                      )

In [ ]:
principais_produtos

Faça as operações correspondente para encontrar os principais destinos (pelo valor total, somando-se todos os produtos) das exportações brasileiras.

In [ ]:
principais_destinos = (exp_2022
                      .groupby(['co_pais'], as_index = False)
                      .agg({'vl_fob': 'sum'})
                      .merge(nome_pais, on = 'co_pais')
                      .sort_values('vl_fob', ascending = False)
                      .loc[:, ['co_pais',  'no_pais', 'vl_fob']]
                      )

In [ ]:
principais_destinos

Agora vamos detalhar as principais produtos exportados por destino.

Crie um DataFrame que liste, para cada país de destino, apenas os **três** principais produtos (Capítulo SH) exportados. 

In [ ]:
principais_produtos_destinos = (exp_2022
                               .groupby(['co_pais', 'co_sh2'], as_index = False)
                               .agg({'vl_fob': 'sum'})
                               .merge(nome_pais, on = 'co_pais')
                               .merge(nome_sh, on = 'co_sh2')
                               .groupby('no_pais', as_index = False, sort = False)
                               .apply(lambda x: x.sort_values('vl_fob', ascending = False).head(3))
                               .loc[:, ['no_pais', 'no_sh2_por', 'vl_fob']]
                               .reset_index(drop = True)
                                )

In [ ]:
principais_produtos_destinos

In [ ]:
principais_produtos_destinos[principais_produtos_destinos['no_pais'] == 'Estados Unidos']

# Explorando as estatísticas de gênero do Banco Mundial

Vamos carregar novamente as séries de dados sobre participação na força de trabalho da população masculina adulta, que vimos na aula 07. A pasta **./data/** já contém os DataFrame que criamos naquela aula. 

O DataFrame **labor_foce_country** apresenta as taxas de participação feminina e masculina, e a diferença (gap) entre as participações, por país, para o ano de 2019.

Você também irá encontrar duas tabelas auxiliares. **country_regions** contém o código e nome de cada país, e o código de sua região geográfica.  **region_names** traz a correspondência entre os códigos e os nomes das regiões.

Neste exercício, você deverá criar um novo DataFrame que apresente, para cada região geográfica, o país com maior *gap* nas taxas de participação na força de trabalho. O DataFrame deverá conter somente as colunas de nome da região, nome do país (que apresentou o maior gap), suas taxas de participação feminina e masculina, e o *gap*.

In [ ]:
import pandas as pd

labor_force_country = pd.read_csv("data/labor_force_country.csv")
country_regions = pd.read_csv("data/country_regions.csv")
region_names = pd.read_csv("data/region_names.csv")

In [ ]:
country_regions_names = country_regions_df.merge(region_names_df, how = "left", on = "region_code")

In [ ]:
biggest_gap_region = (labor_force_country
                     .merge(country_regions_names, 
                            how = "left",
                            left_on="economy",
                            right_on = "country_code")
                     .groupby("region_name", as_index = False)
                     .apply(lambda df: df.sort_values('rate_gap', ascending = False).head(1))
                     .reset_index(drop = True)
                     .loc[:, ["region_name", "country_name", "female_rate", "male_rate", "rate_gap"]]
                     )

In [ ]:
biggest_gap_region

### Funções que geraram e os dados acima e salvaram em .csv (não é necessário rodar)

In [ ]:
import wbgapi as wb

economy_info = wb.economy.info()
region_info = wb.region.info()

country_codes = [pais.get('id') 
                 for pais in economy_info.items
                 if not pais.get('aggregate')]

country_regions_dict = {"country_code": [country.get("id") for country in economy_info.items],
                        "country_name": [country.get("value") for country in economy_info.items],
                        "region_code": [country.get("region") for country in economy_info.items]}

country_regions = pd.DataFrame(country_regions).query("region_code != ''")

regions_dict = {'region_code': [region.get("code") for region in region_info.items],
               'region_name': [region.get("name") for region in region_info.items]}

region_names = pd.DataFrame(regions_dict)

In [ ]:
female_labor_force = wb.data.DataFrame('SL.TLF.ACTI.FE.ZS', time = 2019).squeeze()
male_labor_force = wb.data.DataFrame('SL.TLF.ACTI.MA.ZS', time = 2019).squeeze()

female_labor_force_country = female_labor_force[country_codes]
male_labor_force_country = male_labor_force[country_codes]

labor_force_country = pd.DataFrame({"female_rate": female_labor_force_country,
                                    "male_rate": male_labor_force_country,
                                    "rate_gap": male_labor_force_country - female_labor_force_country})

labor_force_country = labor_force_country.dropna(axis = 0).reset_index()

In [ ]:
country_regions.to_csv("data/country_regions.csv", index_label=False)
region_names.to_csv("data/region_names.csv", index_label=False)
labor_force_country.to_csv("data/labor_force_country.csv", index_label=False)